In [3]:
import tensorflow as tf
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
%matplotlib inline

In [9]:
from models import ANN_model

model=ANN_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)       

In [10]:
# tf.keras.utils.plot_model(model,expand_nested=True)

In [4]:
import base64
model.save_weights('test_weights.h5')

In [5]:
with open('test_weights.h5','rb') as file:
    encoded_string = base64.b64encode(file.read())

In [6]:
with open("test_weights_2.h5","wb") as file:
    file.write(base64.b64decode(encoded_string))

In [7]:
model.load_weights('test_weights_2.h5')

In [19]:
import math

In [8]:
with open('test_weights.h5','rb') as file:
    encoded_string = base64.b64encode(file.read())

In [12]:
len(encoded_string)

6998328

In [21]:
step=len(encoded_string)/1000
step=math.ceil(step)

In [24]:
mylist=[]
for i in range(step):
    msg=encoded_string[i*1000:(i+1)*1000]
    mylist.append(msg)

In [26]:
len(mylist[1:])

6998

In [27]:
new_string=mylist[0]
for i in mylist[1:]:
    new_string=new_string+i

In [28]:
len(new_string)

6998328

In [29]:
with open("test_weights_3.h5","wb") as file:
    file.write(base64.b64decode(new_string))

In [30]:
model.load_weights('test_weights_3.h5')

In [15]:
len(model.layers)

23

In [16]:
for i in range(len(model.layers)):
    w1=model.get_layer(index=i).get_weights()
    print(len(w1)) # 9 non-zero

2
0
2
0
0
0
2
0
2
0
0
0
0
2
4
0
0
2
4
0
0
2
0


In [17]:
i=6
w1=model.get_layer(index=i).get_weights()
w1[0].shape #weight

(3, 3, 32, 64)

In [18]:
w1[1].shape #bias

(64,)

In [19]:
# model.layers[0].set_weights(w1)

### select from weights

In [20]:
weights = model.get_weights()
print(weights[0].shape)  #layer1's weights
print(weights[1].shape)  #layer1's bias
len(weights) #why???? shouldn't it be 18???

(3, 3, 3, 32)
(32,)


22

In [21]:
for i in range(len(weights)):
    print(weights[i].shape)

(3, 3, 3, 32)
(32,)
(3, 3, 32, 32)
(32,)
(3, 3, 32, 64)
(64,)
(3, 3, 64, 64)
(64,)
(2304, 512)
(512,)
(512,)
(512,)
(512,)
(512,)
(512, 100)
(100,)
(100,)
(100,)
(100,)
(100,)
(100, 10)
(10,)


In [22]:
len(weights) ## since 

22

### Set only the final layer trainable

In [23]:
model.trainable = False

for i in range(3):
    model.layers[-(i+1)].trainable = True

## hhh

In [1]:

import tensorflow as tf
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()  #50000 train, 10000 test

category_index=[]     # get a list of turples, each turple is the set of a class i.e. total_set[class_nb][img/label]

for i in range(10):
    index0 = np.where(y_test == i)
    index = index0[0]
    # np.random.shuffle(index)
    category_index.append(index)


##############################################################################
'''
Scenario 5c.10w.10000
Here get 10 worker nodes.
worker: 5 class * 200 class_data  =1000

'''
index_list=[]
worker_nb=10
class_nb=5
class_data=200

class_set=[]
for i in range(worker_nb):
    x=[i+j for j in range(class_nb)]
    for c in range(len(x)):
        while True:
            if x[c]<10:
                break
            x[c]=x[c]-10  
    class_set.append(x)
print(class_set)

record = [0 for i in range(10)]

for j in range(worker_nb):
    index=np.concatenate([category_index[i][record[i]*class_data:record[i]*class_data+class_data] for i in class_set[j]])
    for i in class_set[j]:
        record[i] = record[i]+1
    index_list.append(index)
    print(record)
len(index_list)

[[0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9, 0], [7, 8, 9, 0, 1], [8, 9, 0, 1, 2], [9, 0, 1, 2, 3]]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 2, 2, 2, 2, 1, 0, 0, 0, 0]
[1, 2, 3, 3, 3, 2, 1, 0, 0, 0]
[1, 2, 3, 4, 4, 3, 2, 1, 0, 0]
[1, 2, 3, 4, 5, 4, 3, 2, 1, 0]
[1, 2, 3, 4, 5, 5, 4, 3, 2, 1]
[2, 2, 3, 4, 5, 5, 5, 4, 3, 2]
[3, 3, 3, 4, 5, 5, 5, 5, 4, 3]
[4, 4, 4, 4, 5, 5, 5, 5, 5, 4]
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


10

In [2]:
import collections
a =np.concatenate(index_list)
b =np.unique(a, return_counts=True)

In [3]:
b[1].max()

1